In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('../Datasets/icrisat.xlsx')

In [3]:
df.head()

,Year,State,Crop,Cost
0,1967,Chhattisgarh,rice,110.0
1,1968,Chhattisgarh,rice,98.8
2,1969,Chhattisgarh,rice,94.0
3,1970,Chhattisgarh,rice,95.4
4,1971,Chhattisgarh,rice,95.9


In [4]:
df=df[['State','Crop','Year','Cost']]

In [5]:
df.head()

,State,Crop,Year,Cost
0,Chhattisgarh,rice,1967,110.0
1,Chhattisgarh,rice,1968,98.8
2,Chhattisgarh,rice,1969,94.0
3,Chhattisgarh,rice,1970,95.4
4,Chhattisgarh,rice,1971,95.9


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112805 entries, 0 to 112804
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   State   112805 non-null  object 
 1   Crop    112805 non-null  object 
 2   Year    112805 non-null  int64  
 3   Cost    112805 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 3.4+ MB


In [7]:
df.corr()

,Year,Cost
Year,1.000000,0.667275
Cost,0.667275,1.000000


In [8]:
unique_crops = df.Crop.unique()
print(np.sort(unique_crops))

['barley' 'castor' 'chickpea' 'cotton' 'finger millet' 'groundnut'
 'linseed' 'maize' 'pearl millet' 'pigeonpea' 'rape and mustard' 'rice'
 'seasmum' 'sorghum' 'sugarcane' 'wheat']


In [9]:
unique_states = df.State.unique()
print(np.sort(unique_states))

['Andhra Pradesh' 'Assam' 'Bihar' 'Chhattisgarh' 'Gujarat' 'Haryana'
 'Himachal Pradesh' 'Jharkhand' 'Karnataka' 'Kerala' 'Madhya Pradesh'
 'Maharashtra' 'Orissa' 'Punjab' 'Rajasthan' 'Tamil Nadu' 'Telangana'
 'Uttar Pradesh' 'Uttarakhand' 'West Bengal']


In [10]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [11]:
categorical_data = ['State','Crop']

for label in categorical_data:
    print(label ,': ', len(df[label].unique()))

State :  20
Crop :  16


In [12]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
ohe =  OneHotEncoder(sparse=False, drop="first")
ct = ColumnTransformer([('encoder', ohe, [0,1])], remainder = 'passthrough')

In [13]:
X = ct.fit_transform(X)

In [14]:
print(X)

[[0.0 0.0 1.0 ... 0.0 0.0 1967]
 [0.0 0.0 1.0 ... 0.0 0.0 1968]
 [0.0 0.0 1.0 ... 0.0 0.0 1969]
 ...
 [0.0 0.0 0.0 ... 0.0 0.0 2005]
 [0.0 0.0 0.0 ... 0.0 0.0 2006]
 [0.0 0.0 0.0 ... 0.0 0.0 2007]]


In [15]:
df.sample()

,State,Crop,Year,Cost
104680,Rajasthan,sugarcane,2004,1511.0


In [16]:
X_df = pd.DataFrame(X)
X_df.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1967
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1968
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1969
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1970
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1971


In [17]:
dummy_df = pd.get_dummies(df, drop_first= True)

In [18]:
dummy_df.keys()

Index(['Year', 'Cost', 'State_Assam', 'State_Bihar', 'State_Chhattisgarh',
       'State_Gujarat', 'State_Haryana', 'State_Himachal Pradesh',
       'State_Jharkhand', 'State_Karnataka', 'State_Kerala',
       'State_Madhya Pradesh', 'State_Maharashtra', 'State_Orissa',
       'State_Punjab', 'State_Rajasthan', 'State_Tamil Nadu',
       'State_Telangana', 'State_Uttar Pradesh', 'State_Uttarakhand',
       'State_West Bengal', 'Crop_castor', 'Crop_chickpea', 'Crop_cotton',
       'Crop_finger millet', 'Crop_groundnut', 'Crop_linseed', 'Crop_maize',
       'Crop_pearl millet', 'Crop_pigeonpea', 'Crop_rape and mustard',
       'Crop_rice', 'Crop_seasmum', 'Crop_sorghum', 'Crop_sugarcane',
       'Crop_wheat'],
      dtype='object')

In [19]:
oh_enc_df = pd.DataFrame(X, columns=['Assam','Bihar','Chhattisgarh','Gujarat',
                                     'Haryana','Himachal Pradesh','Jharkhand',
                                     'Karnataka','Kerala','Madhya Pradesh',
                                     'Maharashtra','Orissa','Punjab','Rajasthan',
                                     'Tamil Nadu','Telangana','Uttar Pradesh',
                                     'Uttarakhand','West Bengal','castor','chickpea',
                                     'cotton','finger millet','groundnut','linseed','maize',
                                     'pearl millet','pigeonpea','rape and mustard','rice',
                                     'seasmum','sorghum','sugarcane','wheat','Year'])


oh_enc_df.head()

,Assam,Bihar,Chhattisgarh,Gujarat,Haryana,Himachal Pradesh,Jharkhand,Karnataka,Kerala,Madhya Pradesh,...,maize,pearl millet,pigeonpea,rape and mustard,rice,seasmum,sorghum,sugarcane,wheat,Year
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1967
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1968
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1969
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1970
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1971


In [20]:
oh_enc_df.shape

(112805, 35)

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

In [22]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(random_state=0, n_estimators=50)
regr.fit(X_train, y_train)

RandomForestRegressor(n_estimators=50, random_state=0)

In [23]:
print('R2 Score =', regr.score(X_test, y_test)*100)

R2 Score = 94.36293229187493


In [24]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

cross_val_score(RandomForestRegressor(random_state=0, n_estimators=50), X, y, cv=cv)

array([0.94108107, 0.94271898, 0.94211477, 0.94356318, 0.93923828])

In [25]:
import joblib

with open('../Models/price_prediction_icrisat.joblib','wb') as f:
    joblib.dump(regr,f)